# A2C - CartPole
Implementation of the A2C RL Algorithm for the OpenAI's Gym environment CartPole-V1 (not in parallel).

In [2]:
# Import libraries
import numpy as np
import matplotlib.pyplot as pd

import tensorflow as tf

import gym

In [9]:
# state-value function estimator

class V_Model(tf.keras.layers.Layer):
    
    def __init__(self):
        super(V_Model, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units=16, input_shape=[4], activation='sigmoid')
        self.fc2 = tf.keras.layers.Dense(units=8, activation='sigmoid')
        self.out = tf.keras.layers.Dense(units=1, activation='sigmoid')
        
    def call(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return self.out(x)

In [34]:
# policy function estimator

class P_Model(tf.keras.layers.Layer):
    
    def __inti__(self):
        super(P_Model, self).__init__()
        self.fc1 = tf.keras.layers.Dense(units=16, input_shape=[4], activation='sigmoid')
        self.fc2 = tf.keras.layers.Dense(units=8, activation='sigmoid')
        self.out = tf.keras.layers.Dense(units=1, activation='sigmoid')

In [36]:
# set up environment
env = gym.make('CartPole-v1')

# create two networks: one for the state-value, the other one for the poilcy
v_network = V_Model()
p_network = P_Model()

# create optimizer for both networks and and MSE as loss function
v_optimizer = tf.keras.optimizers.Adam()
p_optimizer = tf.keras.optimizers.Adam()
mse = tf.keras.losses.MSE

# initialize empty list to store memory, which will be used to train the network
observations = []

# hyperparameters
n_episodes = 20
discount = 0.92
max_steps = 300

breakpoint()

# train for n episodes
for ith_episode in range(n_episodes):
    
    # reset the env for each new episode and store the starting state
    state = env.reset()
    
    # build the episode for max_steps steps or until the agent fails
    for t in range(max_steps):
        # get the next action from the policy network
        state = np.reshape(state, [1,4])
        action = 1 if p_network(state) >= 0 else 0
        # perform sampled action and store observation
        next_state, reward, terminal, info = env.step(action)
        obersvation.append((state, action, reward))
        # next time step starts from new state
        state = next_state
        
        # terminate episode when agent fails
        if terminal:
            break
            
    
    # after one episode terminated use the oberservations made to train both networks
    
    # accumulated estimated return starts with zero if the agent failed, else the state value of the last state
    estimated_return = 0 if terminal else v_network(np.reshape(state, [1,4]))
    
    
        





--Return--
> <ipython-input-36-64435ecd0a4a>(21)<module>()->None
-> breakpoint()
(Pdb) c


W0303 19:04:55.918580  5480 base_layer.py:1790] Layer p__model_1 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [13]:
int(-0.5)

0

In [32]:
# Set up the environment
env = gym.make('CartPole-v1')

print(env.action_space.sample())

# for _ in range(10):
#     env.reset()
#     for i in range(100):
#         env.render()
#         env.step(env.action_space.sample())
        
# env.close()

0
